In [1]:
## call the main routines
using NBInclude
@nbinclude("core_fun.ipynb");

We fix the parameters for the case of Austria in an object of type Country. The calibrated values for $\alpha$ and $m$ are obtained latter on.

In [2]:
austria = let 
    
    reform = 2
    A = 50. * 12
    Amin = 46. * 12
    Amax = 54. * 12
    
    r = 0.0048 # 5 % annual interest rate, time unit = month
    c0 = 1. # scale parameter of the cost function, normalized to 1
    q = 0.02 ## 1-exp(-12q) should be cluose to 0.22~0.25
    lambdamax = 1/(39*12/52) ## 39 weeks eligibility
    lambdamin = 1/(209*12/52) ## 209 weeks
    
    ### to be updated after the calibration
    alpha, m =0.02, 0.07
 
    uinit = [0.,0.] ## useless parameters
    
    wbar = 1. # normalization
    bUI = 0.6 * wbar  #  Lalive says 0.4 + family allowances
    b0max = 0.7 * bUI # median UA for the median UI, Lalive
    b0min = b0max 
    
    Country(reform,A,Amin,Amax,r,c0,alpha,q,m,lambdamin,lambdamax,uinit,bUI,b0max,b0min,wbar)
end
;

In [3]:
## precompile functions
let 
    a=49.75*12
    B=6.2
    D=15.4
    simu_aus = find_simu(austria,reltol=1e-5) 
    @time fUI(B, a, simu_aus,austria) 
    @time expectedB(a, simu_aus,austria)
    @time integrand_fNE(B, D, a,simu_aus,austria)
    @time fNE(D, a, simu_aus, austria) 
    @time print(expectedD(a, simu_aus,austria))
end

  3.659746 seconds (5.86 M allocations: 295.883 MiB, 5.87% gc time)
  0.514295 seconds (1.26 M allocations: 44.626 MiB, 4.47% gc time)
  0.593218 seconds (650.83 k allocations: 32.026 MiB, 2.07% gc time)
  0.569877 seconds (1.73 M allocations: 53.003 MiB, 3.95% gc time)
3.351408892483491 10.593629 seconds (101.51 M allocations: 1.734 GiB, 4.32% gc time)


# 1) Find $\alpha$ and $m$ for different values of $q$
We take as moments the expected duration before and after the threshold at age 50.

In [4]:
## We use the method of Simpson to compute the integral instead of the quadgk function
## because computing expectedD is already time-consuming
quadSimpson(fun,a,b) = (b-a)/6 * (fun(a) + 4*fun((a+b)/2) + fun(b)) 
function moments_austria(simu::Simu,C::Country)
    expD1 = quadSimpson(x->expectedD(x,simu,C) ,12*49.75, 12*49.99) / (12*49.99-12*49.75)
    expD2 = quadSimpson(x->expectedD(x,simu,C) ,12*50.01, 12*50.25) / (12*50.25-12*50.01)
    #expD1 = quadgk(x->expectedD(x,simu,C) ,12*49.75, 12*49.99)[1] / (12*49.99-12*49.75)
    #expD2 = quadgk(x->expectedD(x,simu,C) ,12*50.01, 12*50.25)[1] / (12*50.25-12*50.01)
    #expD1 = expectedD(49.75 *12,simu,C)
    #expD2 = expectedD(50.25 *12,simu,C)
    return([expD1, expD2])
end

moments_target = [15.2*12/52, 29.6*12/52]
println(moments_target)

function distance(m1,m2)
    out = 0
    for i in 1:length(m1)
        out += (m1[i] - m2[i]) ^2
    end
    return(out)
end

[3.50769, 6.83077]


distance (generic function with 1 method)

We calibrate our model to determine the values of $\alpha$ and $m$ for different values of $q$.

In [5]:
## generate a Country object with new values for alpha, m and q
function update_Country(alpha,m,q, C::Country)
    Country(C.reform,C.A,C.Amin,C.Amax,C.r,C.c0,
        alpha, q, m,C.lambdamin,C.lambdamax,
        C.uinit,C.bUI,C.b0max,C.b0min,C.wbar)
end

## compute the objective to minimise
function objective_aus(alpha,m,q,C::Country)
    newC = update_Country(alpha,m,q,C)

    ## simulate
    simu = find_simu(newC,reltol=1e-5)
    moments_emp = moments_austria(simu,newC)
    
    IJulia.clear_output(true)
    println(alpha," ",m)
    println(moments_emp)
    
    distance(moments_emp, moments_target) 
end

function find_austria(q,C::Country)
    opt = Opt(:LN_BOBYQA, 2)
    lower_bounds!(opt, [0.005 , 0.])
    upper_bounds!(opt, [2. , 1.])
    xtol_rel!(opt,1e-8)
    stopval!(opt,1e-8)
    min_objective!(opt, (v,g)-> objective_aus(v[1],v[2],q,C))

    init = [0.02, 0.07]

    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence ", ret, ", the value of the objective is ",minf)
    println(minx[1], "  ",minx[2])
    return(minx)
end

q_vec = [0.005, 0.01, 0.025, 0.05, 0.1]
am_vec = zeros(2,5)
for (i,q) in enumerate(q_vec)
    am_vec[:,i] =  find_austria(q,austria)
end
am_vec

0.0770454817057752 0.2822451858703634
[3.5077, 6.83074]
Param found with convergence STOPVAL_REACHED, the value of the objective is 1.0832432757462314e-9
0.0770454817057752  0.2822451858703634


2×5 Array{Float64,2}:
 0.00783389  0.0117762  0.0233762  0.0420305  0.0770455
 0.0268832   0.0407797  0.0826095  0.151536   0.282245 

# 2) Statistics for each value of $q$

In [6]:
jfrUI_bef_vec = zeros(length(q_vec))
jfrUI_lmax_aft_vec = zeros(length(q_vec))
jfrUI_lmin_aft_vec = zeros(length(q_vec))
mom_vec = zeros(length(moments_target),length(q_vec))

for (i,q) in enumerate(q_vec)
    newC = update_Country(am_vec[1,i], am_vec[2,i], q, austria)
    simu_aus = find_simu(newC,reltol=1e-5)
    jfrUI_bef_vec[i] = JFrate(simu_aus.Xbar2(49*12),newC)
    jfrUI_lmax_aft_vec[i] = JFrate(simu_aus.Ybar[2],newC)
    jfrUI_lmin_aft_vec[i] = JFrate(simu_aus.Ybar[3],newC)

    mom_vec[:,i] = moments_austria(simu_aus,newC)
end

mom_vec
mid_vec = vcat(jfrUI_bef_vec',jfrUI_lmax_aft_vec',jfrUI_lmin_aft_vec')

3×5 Array{Float64,2}:
 0.218311  0.216655  0.212039  0.205303  0.194679
 0.220235  0.220442  0.221077  0.222094  0.224044
 0.130074  0.130076  0.130076  0.130078  0.130081

# 3) RDD for each value of $q$

In [7]:
coef_vec = zeros(length(q_vec))
late_vec = zeros(length(q_vec))

## simulated data, monthly
age_grid = 46:(1/12):54
for (i,q) in enumerate(q_vec)
    newC = update_Country(am_vec[1,i], am_vec[2,i], q, austria)
    simu_aus = find_simu(newC,reltol=1e-5)
 
    #Y = [quadgk(x->expectedD(x,simu_aus,newC) ,m,m+1)[1] for m in 12*age_grid] 
    Y = [quadSimpson(x->expectedD(x,simu_aus,newC) ,m,m+0.999999) for m in 12*age_grid] ./0.999999
    
    ## linear spline
    TREAT = [a>=austria.A for a in age_grid*12]
    consta = ones(size(age_grid))
    age1 = collect(age_grid*12)
    age1_spl = (age1 .- austria.A) .* TREAT 
    X = hcat(TREAT, consta, age1, age1_spl)
    beta = inv(X' * X) * (X' * Y)
    coef_vec[i] = beta[1] * 52/12

    
    ## simulated effect of the policy at age A
    mu0 = JFrate(simu_aus.Ybar[4],newC)
    muUI = JFrate(simu_aus.Ybar[5],newC) 
    lam = newC.lambdamax
    expDcounter0 = ((mu0-muUI)/((lam+muUI)*(mu0-lam-muUI)) - lam/(mu0*(mu0-lam-muUI)))
    expDcounter1 = expectedD(newC.Amax,simu_aus,newC)
    late_vec[i] = (expDcounter1-expDcounter0)*52/12
end

late_vec

5-element Array{Float64,1}:
 14.311856805971138
 14.224686127175872
 13.979913977132805
 13.6114359325154  
 13.003272884153667

# The final table is

In [8]:
vcat(q_vec',am_vec,mid_vec,14.8 .* ones(length(q_vec))',coef_vec',late_vec')

9×5 Array{Float64,2}:
  0.005        0.01        0.025       0.05        0.1      
  0.00783389   0.0117762   0.0233762   0.0420305   0.0770455
  0.0268832    0.0407797   0.0826095   0.151536    0.282245 
  0.218311     0.216655    0.212039    0.205303    0.194679 
  0.220235     0.220442    0.221077    0.222094    0.224044 
  0.130074     0.130076    0.130076    0.130078    0.130081 
 14.8         14.8        14.8        14.8        14.8      
 14.3557      14.311      14.1855     13.9916     13.6603   
 14.3119      14.2247     13.9799     13.6114     13.0033   